In [45]:
from pydantic import BaseModel, Field

from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
import re

In [46]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("OPENAI_KEY")

In [47]:
class DocumentInput(BaseModel):
    question: str = Field()


llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=api_key)
# Highlights_of_Morocco 
tools = []
files = [
    # https://abc.xyz/investor/static/pdf/2023Q1_alphabet_earnings_release.pdf
    {
        "name": "Best_of_Cambodia_Northern_Thailand",
        "path": "Best_of_Cambodia_Northern_Thailand.txt",
    },
    # https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q1-2023-Update
    {
        "name": "Cambodia_to_Vietnam_Night_Markets_Noodle-Making",
        "path": "Cambodia_to_Vietnam_Night_Markets_Noodle-Making.txt",
    },
    {
        "name": "Highlights_of_Morocco",
        "path": "Highlights_of_Morocco.txt",
    },
]

for file in files:
    loader = TextLoader(file["path"])
    pages = loader.load_and_split()
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(pages)
    embeddings = OpenAIEmbeddings(openai_api_key=api_key)
    retriever = FAISS.from_documents(docs, embeddings).as_retriever()

    # Wrap retrievers in a Tool
    tools.append(
        Tool(
            args_schema=DocumentInput,
            name=file["name"],
            description=f"useful when you want to answer questions about {file['name']} itinerary",
            func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever),
        )
    )

In [48]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [49]:
import langchain

langchain.debug = True

In [50]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    openai_api_key=api_key
)

agent = initialize_agent(
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,
)

response = agent({"input": "which itinerary is better for visiting these interests: Angkor Wat, foodie?"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "which itinerary is better for visiting these interests: Angkor Wat, foodie?"
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI assistant.\nHuman: which itinerary is better for visiting these interests: Angkor Wat, foodie?"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [5.42s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "function_call": {
                "name": "tool_selection",
               

[llm/end] [1:chain:AgentExecutor > 3:tool:Best_of_Cambodia_Northern_Thailand > 4:chain:RetrievalQA > 6:chain:StuffDocumentsChain > 7:chain:LLMChain > 8:llm:ChatOpenAI] [3.58s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "As an AI, I don't have personal experiences or interests. However, I can provide information about the \"Best of Cambodia & Northern Thailand\" tour if you're interested.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "As an AI, I don't have personal experiences or interests. However, I can provide information about the \"Best of Cambodia & Northern Thailand\" tour if you're interested.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
 

[llm/end] [1:chain:AgentExecutor > 9:tool:Cambodia_to_Vietnam_Night_Markets_Noodle-Making > 10:chain:RetrievalQA > 12:chain:StuffDocumentsChain > 13:chain:LLMChain > 14:llm:ChatOpenAI] [4.09s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "As an AI, I don't have personal experiences or interests. However, I can provide information about the \"Cambodia to Vietnam: Night Markets & Noodle-Making\" tour if you're interested.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "As an AI, I don't have personal experiences or interests. However, I can provide information about the \"Cambodia to Vietnam: Night Markets & Noodle-Making\" tour if you're interested.",
            "additional_kwargs": 

In [51]:
print(response['output'])

If you are interested in exploring both Cambodia and Thailand, the "Best of Cambodia & Northern Thailand" itinerary would be a great choice. It allows you to visit Angkor Wat in Cambodia and also explore the highlights of Northern Thailand.

On the other hand, if you are more interested in experiencing night markets and noodle-making, the "Cambodia to Vietnam: Night Markets & Noodle-Making" itinerary would be a better option. This itinerary takes you through the vibrant night markets of Cambodia and Vietnam, where you can indulge in delicious street food and learn how to make traditional noodles.

Both itineraries offer unique experiences and cater to different interests. It ultimately depends on what you prioritize more - exploring multiple countries or focusing on specific food experiences.
